# In this part we will future explore the dataset

## Import libs

In [4]:
import pandas as pd

## load datasets

Load ad exposure log, Load ad_merge and test

In [5]:
df_exposure = pd.read_hdf('store_test.h5',key = 'df')

In [137]:
df_ad_info = pd.read_hdf('df_ad_merge.h5',key = 'df')

In [7]:
df_test = pd.read_hdf('df_testset.h5',key = 'df')

## understand the data

the target is to get " the exposure rate for the next day", I assume the next day means:"24 hours after the ad is created.

When there is any modification of the ads bid/crowd/expo time, the ad should be treated as another ad.

if the ad is modified within 24 hours, there is no way to estimate the exposure, we should discard the record.

I don't think this is a typical time series prediction problem.

### cate 1: the ad never changed during the period of dataset

In [138]:
def exposure_split_nochange(df_exposure,df_ad_info):
    # first clean the df_exposure
    # before drop has 6005879 records
    expo = df_exposure.drop_duplicates(subset=['AdRequestId','Id'],keep = 'last')
    # after drop has 5952533 records
    # for these ads, no modification has done, so they can be directly used.
    nochg = df_ad_info[df_ad_info['Date'].isnull()]
    nochg.rename(columns={'Date_x':'Date_create',
                          'Date':'Date_p24h'},inplace=True)
    temp_hours = pd.to_datetime(nochg['Date_create'],unit='s') + pd.DateOffset(hours=24)
    nochg['Date_p24h'] = temp_hours.values.astype(np.int64) // 10**9
    expo_nochg = expo[expo['Id'].isin(set(nochg['Id']))].sort_values(['Id','Time'])
    expo_nochg = pd.merge(nochg.loc[:,['Id','Date_p24h']],expo_nochg,how='right',on='Id')
    expo_nochg_rate = expo_nochg.loc[expo_nochg['Time'] < expo_nochg['Date_p24h'],:]. \
                    groupby('Id').count().loc[:,'Time'].rename('exporate')
    df_merge1 = pd.merge(nochg,expo_nochg_rate,how='outer',on='Id')
    #df_id_expo = expo_nochg.groupby(['Id','day']).count().groupby('Id').mean().loc[:,'Time']
    # add daily exposure rate to the ad_info table
    #df_ad_info_nochg['expo_rate'] = df_id_expo.to_list()
    #df_ad_info_nochg[df_ad_info_nochg['Date_x'] == 0]
    return expo_nochg,df_merge1,nochg

In [139]:
expo_nochg,df_merge_nochg,df_ad_info_nochg = exposure_split_nochange(df_exposure,df_ad_info)

/Users/yaoyao/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Users/yaoyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


### ads changed, but has no more changes within 24 hours.

In [246]:
def exposure_split_change(df_exposure,df_ad_info):
    # first clean the df_exposure
    # before drop has 6005879 records
    expo = df_exposure.drop_duplicates(subset=['AdRequestId','Id'],keep = 'last')
    # after drop has 5952533 records
    # for these ads, no modification has done, so they can be directly used.
    chg = df_ad_info[~df_ad_info['Date'].isnull()]
    chg['Date_sft1'] = chg['Date'].shift(-1)
    # for each ed id, keep the last edit
    chg_last = chg.drop_duplicates(subset='Id',keep='last')
    temp_hours = pd.to_datetime(chg_last['Date'],unit='s') + pd.DateOffset(hours=24)
    chg_last['Date_sft1'] = temp_hours.values.astype(np.int64) // 10**9
    # check if ad has been modified in 24 hours
    # for these first record of each id, dateshfi = date_x
    # below the date_shift is correct.
    temp2 = chg[~chg.index.isin(chg_last.index)]
    chg_others = temp2.loc[(temp2['Date_sft1'].astype(int) - temp2['Date'].astype(int)) >= 86400,:]
    temp_hours = pd.to_datetime(chg_others['Date'],unit='s') + pd.DateOffset(hours=24)
    chg_others['Date_sft1'] = temp_hours.values.astype(np.int64) // 10**9
    merge = pd.concat([chg_last,chg_others]).sort_values(['Id','Date'])
    # reformat a bit
    merge = merge.drop(columns='Date_x').rename(columns={'Date_sft1':'Date_p24h','Date':'Date_create'})
    expo_chg = expo[expo['Id'].isin(set(merge['Id']))].sort_values(['Id','Time'])
    f = pd.merge(merge,expo_chg,on='Id').rename(columns={'Time_y':'Time'})
    ff = f[(f['Time'].astype(int) >= f['Date_create'].astype(int)) & \
           (f['Time'].astype(int) < f['Date_p24h'].astype(int))]
    fff = ff.pivot_table(index=['Id','Date_create','Date_p24h'],values='Bid_x',aggfunc=["count"]).reset_index()
    fff.columns = fff.columns.get_level_values(0)
    df_merge_chg = pd.merge(merge,fff,how='left',on=['Id','Date_create','Date_p24h'])
    return expo_chg, df_merge_chg, merge

In [247]:
expo_chg, df_merge_chg, df_ad_info_chg = exposure_split_change(df_exposure,df_ad_info)

/Users/yaoyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/yaoyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/yaoyao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

## merge to training

In [251]:
def merge_training(df_merge_chg,df_merge_nochg):
    # merge three datasets
    df_merge_chg = df_merge_chg.rename(columns={'count':'exporate'})
    merge = pd.concat([df_merge_chg,df_merge_nochg],sort=False)
    return merge

final_merge = merge_training(df_merge_chg,df_merge_nochg)
    

In [262]:
final_merge['Date_create'] = final_merge['Date_create'].astype(int)
final_merge['CommodityId'] = final_merge['CommodityId'].astype(int)
final_merge['AdSize'] = final_merge['AdSize'].astype(int)


In [264]:
final_merge.to_csv('train_final.csv')